In [1]:
import pandas as pd

In [43]:
property_df = pd.read_csv("../data/curated/property_isochrones.csv")
crime_df = pd.read_csv("../data/curated/crime_data.csv")

In [64]:
crime_df_2021 = crime_df[crime_df['Year'] == 2021]

In [65]:
crime_df_2021 = crime_df_2021.rename(columns={'SA2_codes': 'SA2_CODE_2021'})
crime_df_2021 = crime_df_2021.drop(['Year', 'Postcode'], axis = 1)

In [70]:
crime_df_2021 = crime_df_2021.drop_duplicates(subset=['SA2_CODE_2021'], keep='first')

In [71]:
combined_df = pd.merge(crime_df_2021, property_df, on = 'SA2_CODE_2021')

In [72]:
# move offence count and SA2 codes to end of the df
cols = list(combined_df.columns.values)
cols.pop(cols.index('SA2_CODE_2021'))
cols.pop(cols.index('Offence Count'))
combined_df = combined_df[cols+['Offence Count', 'SA2_CODE_2021']]

In [73]:
# instances with SA2 codes that are not inside the crime data
missing = property_df.loc[~property_df['SA2_CODE_2021'].isin(combined_df['SA2_CODE_2021'])].copy()

# fill missing values with average crime rate
avg_crime_rate = crime_df_2021['Offence Count'].mean().round()
missing['Offence Count'] = avg_crime_rate

In [74]:
# move SA2 codes to end of the df
cols = list(missing.columns.values)
cols.pop(cols.index('SA2_CODE_2021'))
missing = missing[cols+['SA2_CODE_2021']]

In [75]:
combined_df = pd.concat([missing, combined_df])

In [76]:
combined_df.shape

(10864, 20)

In [77]:
combined_df.to_csv('../data/curated/full_data.csv', index=False)